In [ ]:
 \label{fig:coding}

#This code is prepared to classify and recognize ship s on the
#UAV generated images
#Importing the basic libraries and modules
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator , load_img
from keras.callbacks import ModelCheckpoint , EarlyStopping , CSVLogger
from keras.applications import VGG16
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras import models
from keras import layers
from keras.layers import Dense , Flatten
from keras import optimizers
import pandas as pd
import time

start_time = time.time() #Recording the time starts here
#Defining the variables by assigning their path to the directories
train_dir = ’/Thesis_Dataset/Train’
validation_dir = ’/Thesis_Dataset/validate’
test_dir = ’/Thesis_Dataset/Test’

eval_dir = ’/hesis_Dataset/test_evaluation’

image_size = 224

#Load the VGG model , remember to assign false on top layers to remove it and
#avoid the original 1000 classes of ImageNet to modify on our classes (i.e.5)
vgg_conv = VGG16(weights=’imagenet’, include_top=False , input_shape=
(image_size , image_size , 3))
vgg_conv.summary()

# Freeze/Unfreeze the layers
# Freeze the layers: No specific rule (More as heat and trial)
#but early layers are normally freezed (like edges ,)
#whereas latter layers normally extract the specific \\properties of dataset.
#for layer in vgg_conv.layers[:]: # No layers are freezed in this case
# layer.trainable = True

# Check the trainable status of the individual layers
#for layer in vgg_conv.layers:
# print(layer , layer.trainable)

# Create the model
model = models.Sequential()
# Add the vgg convolutional base model
model.add(vgg_conv)
# Add new layers
#Dropout value can be changed or removed
#BatchNormalization can be added; it is considered good for robustness
model.add(layers.Flatten())
model.add(layers.Dense(512, activation=’relu’))
#model.add(BatchNormalization())
model.add(layers.Dropout(0.5))
#model.add(layers.Dense(512, activation=’relu’))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation=’softmax’))
#Add a layer where input is the output of the second last layer
#x = vgg_conv.output
#x = Flatten()(x)
#predictions = Dense(5, activation=’softmax’)(x)
#model = Model(inputs=vgg_conv.input , outputs=predictions)
#model.summary()

#Data augmentation done here, these values can be changed.
train_datagen = ImageDataGenerator(rescale=1./255)
"""
rotation_range=2,
width_shift_range=0.2,
height_shift_range=0.2,
horizontal_flip=True,
fill_mode=’nearest’)
"""

test_datagen = ImageDataGenerator(rescale=1./255)

# Change the batchsize according to your system RAM
train_batchsize = 32
val_batchsize = 16
eval_batchsize=1
# Data Generator for Training data
train_generator = train_datagen.flow_from_directory(
train_dir ,
target_size=(image_size , image_size),
batch_size=train_batchsize ,
class_mode=’categorical’,
shuffle=True)

# Data Generator for Validation data
validation_generator = test_datagen.flow_from_directory(
validation_dir ,
target_size=(image_size , image_size),
batch_size=val_batchsize ,
class_mode=’categorical’,
shuffle=False)
# Compile the model
##Activation Functions like ReLU, Tanh, LeakyReLU are available
model.compile(loss=’categorical_crossentropy’,
optimizer=optimizers.Adam(lr=1e-4),
metrics=[’acc’])
# This is to save the model according to the conditions and
#stops if validation accuracy is not improving as per the assigned condition
checkpoint = ModelCheckpoint("VGglast.h5", monitor=’val_acc’,
verbose=1, save_best_only=True, save_weights_only=False , mode=’auto’,
period=1)
early = EarlyStopping(monitor=’val_acc’, min_delta=0, patience=10,
verbose=1, mode=’auto’)
csv_logger = CSVLogger("vgglast.csv", append=True)
# Train the Model
history = model.fit_generator(
train_generator ,
steps_per_epoch=train_generator.samples//train_generator.batch_size ,
epochs=20 ,
validation_data=validation_generator ,
validation_steps=validation_generator.samples//
validation_generator.batch_size ,
callbacks = [checkpoint , early , csv_logger])

# Save the Model
#model.save(’last_elu4lyr_frze.h5’)
import pickle
with open(’vgglast’, ’wb’) as handle: # saving the history of the model
pickle.dump(history.history , handle)

print("%fseconds" % (time.time() - start_time))
#from keras.utils import plot_model
#plot_model(model , to_file=’model.png’)
# Plot the accuracy and loss curves
acc = history.history[’acc’]
val_acc = history.history[’val_acc’]
loss = history.history[’loss’]
val_loss = history.history[’val_loss’]

epochs = range(len(acc))

plt.plot(epochs , acc, ’b’, label=’Trainingacc’)
plt.plot(epochs , val_acc , ’r’, label=’Validationacc’)
plt.title(’Trainingandvalidationaccuracy’)
plt.legend()
plt.savefig(’vgglast_acc.png’)
plt.figure()

plt.plot(epochs , loss, ’b’, label=’Trainingloss’)
plt.plot(epochs , val_loss , ’r’, label=’Validationloss’)
plt.title(’Trainingandvalidationloss’)
plt.legend()
plt.savefig(’vgglast_loss.png’)
plt.show()
#plt.savefig(["plot2.png"])
#Code below is to predict the validation accuracy and
#then to obtain the confusion matrix. Depending upon accuracy on the
#dataset we need we should use the related dataset generator.
# Below is the prediction for validation data, we can switch it for
# test dataset
# as well by copying the code from lines created below for test generator)
#and changing the validation_generator with test_generator

#At first , evaluating the validation data! Should it be test data??
start_time = time.time()
evaluation_generator = test_datagen.flow_from_directory(
    eval_dir ,
    target_size=(image_size , image_size),
    batch_size=eval_batchsize ,
    class_mode=’categorical’,
    shuffle=False)

scores = model.evaluate_generator(evaluation_generator , steps = 596)
print(’Loss:’, scores[0])
print(’Accuracy:’, scores[1])

print("%fseconds" % (time.time() - start_time))

start_time = time.time()

#For predicting accuracy
179 # Get the filenames from the generator
180 fnames = evaluation_generator.filenames
181
182 # Get the ground truth from generator
183 ground_truth = evaluation_generator.classes
184
185 # Get the label to class mapping from the generator
186 label2index = evaluation_generator.class_indices
187
188 # Getting the mapping from class index to class label
189 idx2label = dict((v,k) for k,v in label2index.items())
190
191 # Get the predictions from the model using the generator
192 predictions = model.predict_generator(evaluation_generator ,
193 steps=evaluation_generator.samples/
194 evaluation_generator.batch_size ,verbose=1)
195 pred_class = np.argmax(predictions ,axis=1)
196
197 errors = np.where(pred_class != ground_truth)[0]
198 print("Nooferrors={}/{}".format(len(errors),eval_generator.samples))
199 """
200 #Showtheerrors
201 foriinrange(len(errors)):
202 pred_class=np.argmax(predictions[errors[i]])
203 pred_label=idx2label[pred_class]
204
205 title=’Originallabel:{},Prediction:{},confidence:{:.3f}’.format(
206 fnames[errors[i]].split(’/’)[0],
207 pred_label ,
208 predictions[errors[i]][pred_class])
209
210 original=load_img(’{}/{}’.format(validation_dir ,fnames[errors[i]]))
211 plt.figure(figsize=[7,7])
212 plt.axis(’off’)
213 plt.title(title)
214 plt.imshow(original)
215 plt.show()
216 """
217 print("%fseconds" % (time.time() - start_time))
218
219 #To save the prediction in excel sheet but since I am directly predicting
220 #here Confusion matrix , it was not saved , may be useful in server to save it.
221 import sklearn.metrics as metrics
222 #pred_class = np.argmax(prob, axis=1)
223 target_names = [’cargo’, ’no_ship’, ’ship_dinghies’,
224 ’small_boat’, ’small_boat_patrol’]
225 report = metrics.classification_report(ground_truth ,
226 pred_class , target_names=target_names)
227 print(report)

import itertools
230 #cm = confusion_matrix(idx2label , predictions )
231 def plot_confusion_matrix(cm, classes ,
232 normalize=False ,
233 title=’Confusionmatrix’,
234 cmap=plt.cm.Blues):
235 if normalize:
236 cm = cm.astype(’float’) / cm.sum(axis=1)[:, np.newaxis]
237 print("Normalizedconfusionmatrix")
238 else:
239 print(’Confusionmatrix ,withoutnormalization’)
240
241 #print(cm)
242
243 plt.imshow(cm, interpolation=’nearest’, cmap=cmap)
244 plt.title(title)
245 plt.colorbar()
246 tick_marks = np.arange(len(classes))
247 plt.xticks(tick_marks , classes , rotation=45)
248 plt.yticks(tick_marks , classes)
249
250 fmt = ’.2f’ if normalize else ’d’
251 thresh = cm.max() / 5.
252 for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
253 plt.text(j, i, format(cm[i, j], fmt),
254 horizontalalignment="center",
255 color="white" if cm[i, j] > thresh else "black")
256
257 plt.ylabel(’Truelabel’)
258 plt.xlabel(’Predictedlabel’)
259 plt.tight_layout()
260
261
262 # Compute confusion matrix
263 cnf_matrix = confusion_matrix(ground_truth , pred_class)
264 np.set_printoptions(precision=2)
265 # Plot normalized confusion matrix
266 plt.figure()
267 plot_confusion_matrix(cnf_matrix , classes=target_names , normalize=False ,
268 title=’Confusionmatrix’)
269
270 plt.show()
271 plt.savefig(’vgglast.png’)
272
273 print("%fseconds" % (time.time() - start_time))
274
275
276
277 start_time = time.time()

#To evaluate accuracy of the model with the test data
279 test_generator = test_datagen.flow_from_directory(
280 test_dir ,
281 target_size=(image_size , image_size),
282 batch_size=1,
283 class_mode=None,
284 shuffle=False)
285
286
287 test_generator.reset()
288 #Predicting for the test data
289
290 pred=model.predict_generator(test_generator , steps=
291 test_generator.samples//test_generator.batch_size )
292
293 predicted_class_indices=np.argmax(pred, axis=1)
294 labels = (train_generator.class_indices)
295 labels=dict((v,k) for k,v in labels.items())
296 predictions = [labels[k] for k in predicted_class_indices]
297
298 filenames=test_generator.filenames
299 results=pd.DataFrame({"Filename":filenames ,
300 "Predictions":predictions})
301 results.to_csv("VGglast.csv", index=True)
302
303 print("%fseconds" % (time.time() - start_time))